In [0]:
import sys
from datetime import timedelta
from pyspark.sql.functions import col, current_timestamp

workspace = '/Workspace/Shared/eixo5_grupo2_20252/projeto/processamento'
sys.path.insert(0, workspace)

import schemas.schemas_bronze as sb

In [0]:
landing = '/Volumes/landing/raw/sistema_recomendacao/'
catalogo = 'filmes'
schema = 'bronze'

for f in dbutils.fs.ls(landing):
    df = spark.read.csv(f.path, header=True, schema=eval(f"sb.{f.name[:-4]}"))
    
    df = df.withColumn('data_ingestao', current_timestamp() - timedelta(hours=3))

    (df.write
        .mode('overwrite')
        .format('delta')
        .option('optimizeWrite', 'true')
        .option('autoCompact', 'true')
        .option('mergeSchema', 'true')
        .saveAsTable(f'{catalogo}.{schema}.{f.name[:-4]}')
    )